In [ ]:
import numpy as np
import pennylane as qml
from pennylane import numpy as np
from pennylane.templates import RandomLayers,BasicEntanglerLayers
import qiskit

In [ ]:
T=np.load('data.npy') ##### Time series data. Should be a tensor of shape: (batch, time, features)

In [ ]:
n_w = 4 # numbers of wires def 4
dev = qml.device("default.qubit", wires= n_w)

In [ ]:
### Quantum Circuit

n_layers = 2

# Random circuit parameters
rand_params = np.random.uniform(high= 2 * np.pi, size=(n_layers, n_w)) # def 2, n_w = 4

@qml.qnode(dev)
def circuit(phi=None):
    
    for j in range(n_w):
        qml.RY(np.pi * phi[j], wires=j)

    qml.BasicEntanglerLayers(weights=rand_params, wires=range(n_w))
    return [qml.expval(qml.PauliZ(wires=i)) for i in range(n_w)]


In [ ]:
## Piece-wise Encoding
def encode(feature,kr=4):
    out=[]
    for j in range(0,feature.shape[0]-3,4):
        q_results = circuit(
                [feature[j], feature[j+1], feature[j + 2], feature[j + 3]]
            )
        out.append(q_results)
    return out

In [ ]:
Encoded_data=[]
from tqdm import tqdm
for j in tqdm(range(0,len(T))):
    
    x=T[j,:,:]
    F=[]
    F1=[]
    for i in range(0,x.shape[1]):
        
        feature=x[:,i]
        E=encode(feature)
        encoded=np.hstack(E)  
        F1.append(encoded) 
        
    F1=np.stack(F1) # Encoded time series
    Encoded_data.append(F1)
 
      
   

In [ ]:
Encoded_data=np.stack(Encoded_data)
Encoded_data=np.rollaxis(Encoded_data,2,1)  # (batch, time,features)